In [36]:
import pandas as pd
import fsspec
from google.oauth2 import service_account

import dask.dataframe as dd
import xarray as xr
from shapely import wkb
import geopandas as gpd
from xclim.indices.stats import standardized_index_fit_params
from xclim.indices import standardized_precipitation_index

import xesmf as xe
# Path to your service account key file
service_account_json =  f"coiled-data-key.json"
# Create credentials object
credentials = service_account.Credentials.from_service_account_file(
    service_account_json,
    scopes=["https://www.googleapis.com/auth/devstorage.read_only"],
  # Adjust scopes if needed
)

In [5]:
# GCS file URL
gcs_file_url = 'gs://seas51/ea_admin0_2_custom_polygon_shapefile_v5.parquet'
ddf = dd.read_parquet(gcs_file_url, storage_options={'token': credentials}, engine='pyarrow')
fdf = ddf[ddf['gbid'].str.contains('kmj')]
df1 = fdf.compute()
df1['geometry'] = df1['geometry'].apply(wkb.loads)
gdf = gpd.GeoDataFrame(df1, geometry='geometry')

In [8]:
bounds=gdf.bounds
lat_min = bounds['miny'].min()
lat_max = bounds['maxy'].max()
lon_min = bounds['minx'].min()
lon_max = bounds['maxx'].max()

In [9]:
# GCS path to the Zarr file
gcs_zarr_path = "gs://seas51/chirps_v2_monthly_20241012.zarr"
fs = fsspec.filesystem("gs", token=credentials)
# Get the mapper
m = fs.get_mapper(gcs_zarr_path)
# Open the dataset using xarray
ds = xr.open_zarr(m, consolidated=False)
# Define the polygon boundaries
#lon1, lat1 = 32.0, 1.0
#lon2, lat2 = 36.0, 5.0
# Extract the data for the specified polygon
ds_subset = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
# Compute the data locally
ds_subset_computed = ds_subset.compute()
ds_subset_computed

<xarray.Dataset> Size: 4MB
Dimensions:    (latitude: 58, longitude: 30, time: 524)
Coordinates:
  * latitude   (latitude) float32 232B 1.375 1.425 1.475 ... 4.125 4.175 4.225
  * longitude  (longitude) float32 120B 33.52 33.57 33.62 ... 34.88 34.93 34.97
  * time       (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2024-08-01
Data variables:
    precip     (time, latitude, longitude) float32 4MB 22.12 21.01 ... 3.066
Attributes: (12/15)
    Conventions:       CF-1.6
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    comments:           time variable denotes the first day of the given month.
    creator_email:     pete@geog.ucsb.edu
    creator_name:      Pete Peterson
    date_created:      2024-09-16
    ...                ...
    history:           created by Climate Hazards Group
    institution:       Climate Hazards Group.  University of California at Sa...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    title:             CHIRPS Version 2.0
    version:           Version 2.0
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html

In [10]:
# Calculate the total size of the dataset in bytes
total_size = ds_subset_computed.nbytes
print(f"Total data size: {total_size / 1e6:.2f} MB")

Total data size: 3.65 MB


In [13]:
ch_ds=ds_subset_computed
ch_ds['precip'].attrs['units'] = 'mm/month'
#ch_ds1 = ch_ds.chunk(-1)
aa=ch_ds.precip
spi_3 = standardized_precipitation_index(
     aa,
     freq="MS",
     window=3,
     dist="gamma",
     method="APP",
     cal_start='1991-01-01',
     cal_end='2018-01-01',
    fitkwargs={"floc": 0} 
)

In [14]:
a_s3=spi_3.compute()

In [15]:
a_s3

<xarray.DataArray (time: 524, latitude: 58, longitude: 30)> Size: 7MB
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [ 0.89230498,  1.15567757,  1.19466508, ...,  0.53369031,
          0.50537835,  0.520004  ],
        [ 0.81733088,  0.91038923,  1.03332347, ...,  0.86859292,
          0.59171961,  0.21290168],
        [ 0.80804024,  0.65391863,  0.70077252, ...,  0.79682613,
          0.78945475,  0.69782281]],

       [[ 0.17483145,  0.3058583 ,  0.49464096, ...,  1.01269937,
          1.19279424,  1.24502222],
        [ 0.2441535 ,  0.36439567,  0.48213363, ...,  1.15956962,
          1.25759714,  1.46374045],
        [ 0.30180248,  0.29743274,  0.54564223, ...,  1.18142448,
          1.44488639,  1.48639166],
        ...,
        [ 0.75283627,  0.92055878,  0.91589611, ...,  0.61337703,
          0.58189647,  0.63920539],
        [ 0.65925889,  0.73496359,  0.76249422, ...,  0.98569471,
          0.71312295,  0.232266  ],
        [ 0.69458037,  0.51833478,  0.52091514, ...,  1.00296797,
          0.95398632,  0.82770172]]])
Coordinates:
  * latitude      (latitude) float32 232B 1.375 1.425 1.475 ... 4.175 4.225
  * longitude     (longitude) float32 120B 33.52 33.57 33.62 ... 34.93 34.97
    prob_of_zero  (time, latitude, longitude) float64 7MB 0.0 0.0 ... 0.0 0.0
  * time          (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2024-08-01
Attributes:
    calibration_period:  ('1991-01-01', '2018-01-01')
    freq:                MS
    window:              3
    scipy_dist:          gamma
    method:              APP
    group:               time.month
    units:               1
    time_indexer:        {}

In [16]:
ch_spi= a_s3.to_dataset(name='spi3')

total_size = ch_spi.nbytes
print(f"Total data size: {total_size / 1e6:.2f} MB")

Total data size: 14.59 MB


In [17]:
ch_spi['time']

<xarray.DataArray 'time' (time: 524)> Size: 4kB
array(['1981-01-01T00:00:00.000000000', '1981-02-01T00:00:00.000000000',
       '1981-03-01T00:00:00.000000000', ..., '2024-06-01T00:00:00.000000000',
       '2024-07-01T00:00:00.000000000', '2024-08-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2024-08-01

In [28]:
#ds2=dr_out.to_dataset()
ch_spi.to_netcdf(f'kmj-nr-chirps-v2.0.monthly.nc')

In [20]:
import xarray as xr
import fsspec
from google.oauth2 import service_account

# GCS path to the Zarr file
gcs_zarr_path = "gs://seas51/seas51_20241012_v3.zarr"

# Define path to your credentials JSON file
credentials_path = "coiled-data-key.json"

# Specify the correct GCS scope
scopes = ["https://www.googleapis.com/auth/devstorage.read_write"]

# Create credentials object with the required scope
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=scopes
)

# Create an fsspec filesystem with GCS using the service account
fs = fsspec.filesystem("gs", token=credentials)

# Get the mapper
m = fs.get_mapper(gcs_zarr_path)

# Open the dataset using xarray
sds = xr.open_zarr(m, consolidated=False)

# Extract the data for the specified polygon
sds_subset = sds.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))

# Compute the data locally
sds_subset_computed = sds_subset.compute()

In [21]:
sds_subset_computed

<xarray.Dataset> Size: 4MB
Dimensions:        (forecastMonth: 6, latitude: 3, longitude: 2, number: 51,
                    time: 526)
Coordinates:
  * forecastMonth  (forecastMonth) int64 48B 1 2 3 4 5 6
  * latitude       (latitude) float64 24B 4.0 3.0 2.0
  * longitude      (longitude) float64 16B 34.0 35.0
  * number         (number) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    surface        float64 8B 0.0
  * time           (time) datetime64[ns] 4kB 1981-01-01 ... 2024-10-01
Data variables:
    tprate         (number, forecastMonth, time, latitude, longitude) float32 4MB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2024-10-14T00:13 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [25]:
def apply_spi3(cont_db,lead_val):
    """
    Calculates the 3-month Standardized Precipitation Index (SPI) for a specified lead time.

    Parameters:
    - cont_db (xarray.Dataset): The input dataset containing total monthly precipitation data.
    - lead_val (int): The lead time value for which the SPI is calculated.

    Returns:
    - cont_spi (list): A list of xarray.DataArrays containing the SPI values for each ensemble member.
    """
    lt1_db = cont_db.sel(forecastMonth=lead_val)
    lt1_db['tprate'].attrs['units'] = 'mm/month'
    cont_spi=[]
    for nsl in lt1_db.number.values:
        lt1_db2=lt1_db.sel(number=nsl)
        #lt1_db3 = lt1_db2.chunk({'time': 4, 'latitude': 2, 'longitude': 2})
        #lt1_db3 = lt1_db2.chunk(-1)
        aa=lt1_db2.tprate
        spi_3 = standardized_precipitation_index(
             aa,
             freq="MS",
             window=3,
             dist="gamma",
             method="APP",
             cal_start='1991-01-01',
             cal_end='2018-01-01',
             fitkwargs={"floc": 0} 
        )  
        a_s3=spi_3.compute()
        cont_spi.append(a_s3)
        aa=[]
        lt1_db3 = []
        lt1_db2 = []
        print(nsl)
    return cont_spi

In [ ]:
lead_val=1
cont1=apply_spi3(sds_subset_computed,lead_val)
t1d = xr.concat(cont1, dim='member')
t1da= t1d.to_dataset(name='spi3')

lead_val=2
cont2=apply_spi3(sds_subset_computed,lead_val)
t2d = xr.concat(cont2, dim='member')
t2da= t2d.to_dataset(name='spi3')
print('lt2')

lead_val=3
cont3=apply_spi3(sds_subset_computed,lead_val)
t3d = xr.concat(cont3, dim='member')
t3da= t3d.to_dataset(name='spi3')
print('lt3')

lead_val=4
cont4=apply_spi3(sds_subset_computed,lead_val)
t4d = xr.concat(cont4, dim='member')
t4da= t4d.to_dataset(name='spi3')
print('lt4')

lead_val=5
cont5=apply_spi3(sds_subset_computed,lead_val)
t5d = xr.concat(cont5, dim='member')
t5da= t5d.to_dataset(name='spi3')
print('lt5')

lead_val=6
cont6=apply_spi3(sds_subset_computed,lead_val)
t6d = xr.concat(cont6, dim='member')
t6da= t6d.to_dataset(name='spi3')
print('lt6')

In [27]:
ds_ea = xr.concat([t1d,t2d,t3d,t4d,t5d,t6d], dim='lead')
ds_ea= ds_ea.to_dataset(name='spi3')
ds_ea.spi3.nbytes / (1024*1024)
ds_ea.to_netcdf(f'kmj_nr_seas51_spi3_xclim_20241105.nc')

In [33]:
kn_obs=xr.open_dataset(f'kmj-nr-chirps-v2.0.monthly.nc')
kn_obs

<xarray.Dataset> Size: 15MB
Dimensions:       (latitude: 58, longitude: 30, time: 524)
Coordinates:
  * latitude      (latitude) float32 232B 1.375 1.425 1.475 ... 4.175 4.225
  * longitude     (longitude) float32 120B 33.52 33.57 33.62 ... 34.93 34.97
    prob_of_zero  (time, latitude, longitude) float64 7MB ...
  * time          (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2024-08-01
Data variables:
    spi3          (time, latitude, longitude) float64 7MB ...

In [34]:
ds_p_m1=ds_ea.sel(lead=1)
ds_p_m1

<xarray.Dataset> Size: 3MB
Dimensions:        (latitude: 3, longitude: 2, member: 51, time: 526)
Coordinates:
  * latitude       (latitude) float64 24B 4.0 3.0 2.0
  * longitude      (longitude) float64 16B 34.0 35.0
    forecastMonth  int64 8B 2
    number         (member) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    surface        float64 8B 0.0
    prob_of_zero   (member, time, latitude, longitude) float64 1MB 0.0 ... 0.0
  * time           (time) datetime64[ns] 4kB 1981-01-01 ... 2024-10-01
Dimensions without coordinates: member
Data variables:
    spi3           (member, time, latitude, longitude) float64 1MB nan ... nan

In [38]:
cont_d=[]

for fm in [0,1,2,3,4,5]:
    ds_p_m1=ds_ea.sel(lead=fm)
    ds_out = xr.Dataset(
          {"lat": (["lat"], kn_obs['latitude'].values, {"units": "degrees_north"}),
          "lon": (["lon"], kn_obs['longitude'].values, {"units": "degrees_east"}),})
    gd2=ds_p_m1.rename({'longitude':'lon','latitude':'lat'})
    agd = gd2["spi3"]
    regridder = xe.Regridder(gd2, ds_out, "bilinear")
    dr_out = regridder(agd, keep_attrs=True)
    ds2=dr_out.to_dataset()
    cont_d.append(ds2)
    #monthname=mnl.lower().split('.')[0]
    #ds2.to_netcdf(f'{output_path_location}kmj_25km_lt_month_{fm}.nc')
    
kn_fct = xr.concat(cont_d, dim='lead')
kn_fct=kn_fct.rename({'time':'init','forecastMonth':'lead'})
#ld1 = ld.rename({'time':'init','forecastMonth':'lead'}).set_index(init='time', lead='forecastMonth')
#ld1 = ld.swap_dims({'time': 'init', 'forecastMonth': 'lead'})
kn_fct['lead'].attrs['units'] = 'months'


#output
kn_fct.to_netcdf(f'kmj_5km_seas51_spi3_xclim_20241105.nc')

/tmp/ipykernel_241/3899825425.py:18: UserWarning: rename 'forecastMonth' to 'lead' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  kn_fct=kn_fct.rename({'time':'init','forecastMonth':'lead'})
